# Test Everything, Carefully: Generating Questions from Text
## GA DSI 26 Capstone Project
## Chapter 3: Conclusions

# Scoring Metrics (probably move to conclusions)

There is a great link with explains the most popular scoring metrics [here](https://gist.github.com/shagunsodhani/f05748b6339ceff26420ceecfc79d58d), but the ones we have used are:
- BLEU: co-occurences of n-grams in predicted and ground truth, developed for machine translation
- Cosine similarity: considers the distance between the sentence vectors

Based on the literature, however, these scores do not capture the complexity of a generated question, identified in previous research ([Nema et al, 2018](https://arxiv.org/abs/1808.10192), [Callison-Burch et al, 2006](https://aclanthology.org/E06-1032.pdf) and [Liu et al, 2016](https://arxiv.org/abs/1603.08023v1)) of fluency, adequacy, coherence, and answerability. BLEU (with the related metrics of ROGUE and METEOR) considers n-grams, while cosine similarity considers vectorized embeddings, but these do not nearly capture the required nuances that distinguish a good question.

## Validation Results

| Topic                          | Original Question                                                     | Answer                                                                                                                                     | Generated Question                                                                                                              | Generated Answer                                                                                                                           | BLEU   | Cosine Similarity |
|--------------------------------|-----------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------|--------|-------------------|
| Kenya                          | Who decided not to come visit the country in 2013?                    | US President Barack Obama                                                                                                                  | Who chose not to visit Kenya during his mid-2013 African trip?                                                                  | US President Barack Obama                                                                                                                  | 0.118  | 0.907             |
| Philo of Byzantium             | What did Philo incorrectly assume that the air became?                | fire                                                                                                                                       | In his work Pneumatica, Philo incorrectly surmised that parts of the air in the vessel were converted into what?                | fire                                                                                                                                       | 0.107  | 0.905             |
| The Amazon                     | In which point did the drainage basin of the Amazon split?            | During the mid-Eocene, it is believed that the drainage basin of the Amazon was split along the middle of the continent by the Purus Arch. | When did the Andes Mountains rise?                                                                                              | During the mid-Eocene, it is believed that the drainage basin of the Amazon was split along the middle of the continent by the Purus Arch. | 0.0929 | 0.840             |
| Protestant Reformation         | What were the papal legate's orders from the Pope?                    | arrest                                                                                                                                     | What did Cajetan instruct Luther to do?                                                                                         | arrest                                                                                                                                     | 0.0924 | 0.747             |
| Victoria and Albert Collection | Which South Asian island nation is represented in the V&A collection? | Sri Lanka                                                                                                                                  | Along with Thailand, Burma, Cambodia and Indonesia, what country displays in gold, silver, bronze, stone, terracotta and ivory? | Sri Lanka                                                                                                                                  | 0.0958 | 0.739             |
| ---                            | ---                                                                   | ---                                                                                                                                        | ---                                                                                                                             | ---                                                                                                                                        | ---    | ---               |
| Kenya                          | Who decided not to come visit the country in 2013?                    | [MASK]                                                                                                                                     | Who invited Obama to visit China in July 2015?                                                                                  | President Xi Jinping                                                                                                                       | 0.110  | 0.856             |
| Philo of Byzantium             | What did Philo incorrectly assume that the air became?                | [MASK]                                                                                                                                     | Who conducted one of the first known experiments on the relationship between combustion and air?                                | Philo of Byzantium                                                                                                                         | 0.100  | 0.726             |
| The Amazon                     | In which point did the drainage basin of the Amazon split?            | [MASK]                                                                                                                                     | When was the drainage basin of the Amazon split?'                                                                               | mid-Eocene                                                                                                                                 | 0.073  | 0.333             |
| Protestant Reformation         | What were the papal legate's orders from the Pope?                    | [MASK]                                                                                                                                     | Who was used to reformers and heretics?                                                                                         | Pope Leo X                                                                                                                                 | 0.100  | 0.745             |
| Victoria and Albert Collection | Which South Asian island nation is represented in the V&A collection? | [MASK]                                                                                                                                     | What was made between 500 AD and 2000?                                                                                          | gleaming boxes                                                                                                                             | 0.090  | 0.783             |

From the above, we can see clearly that the scores do not correspond to the interpretability of the question. For example, the masked question for the Protestant Reformation has a better BLEU score than the unmasked one, although nowhere in the passage mentions whether or not Pope Leo X was used to heretics. The masked question on Philo has close scores to that of the unmasked question, although the question is a rather obvious one and has little educational value. It seems that both our masked model and the scoring metrics are flawed, and more work needs to be done on them.

One interesting thing that stands out is the question on the Amazon (for `num_beams=3`, `temperature=0.7`), where the question generated was simply "sin", and the answer was "The Solim", which presumably is short for "The Solimões Basin", but was cut short at the "õ". This would indicate that our model does not recognize non-latin orthography, and we would need to train our model on more data for it to register this. The "sin" question could also be a mistaken lemma of "basin", which we would need to address when refining our model.

Masked generation aside, our unmasked questions seem to fare much better. More information tends to be given in the question, which might help weaker students understand and answer questions better, although they might make the questions too easy for intermediate learners. However, the BLEU scores for the generated questions were very low, which does not tally with a visual inspection. For example, the topic on Kenya has a BLEU score of 0.118 which is a relatively bad score (1 being the best and 0 the worst), however the generated question (arguably) provides slightly more nuance for the respondent to infer the correct answer.

On a visual inspection, the cosine similarity score seems to be a slightly better determiner of how close the generated question is to the target question. However, the process of question generation is not an injective (one-to-one) procedure, as there can be many possible questions for any given context and answer. This cannot be captured by the cosine similarity metric.

## Test Results

| Topic           | Answer                                              | Generated Question                                            | Generated Answer                                                                                       |
|-----------------|-----------------------------------------------------|---------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| Simple Passage  | stale                                               | The issue has been automatically marked as what?              | stale                                                                                                  |
| Bible           | you shall love your neighbor as yourself            | What is the whole law fulfilled in one word?                  | you shall love your neighbor as yourself                                                               |
| Jimi Hendrix    | 'Hey Joe', 'Purple Haze', and 'The Wind Cries Mary' | What three hits did Hendrix have in the UK?                   | 'Hey Joe', 'Purple Haze', and 'The Wind Cries Mary'                                                    |
| Physics Lecture | granular structure of light                         | What was confirmed by the photoelectric effect?               | granular structure of light                                                                            |
| Zhuangzi        | none                                                | How many orifices did Hundun have?                            | none                                                                                                   |
| ---             | ---                                                 | ---                                                           | ---                                                                                                    |
| Simple Passage  | [MASK]                                              | Why has this issue been marked as stale?                      | it has not had recent activity                                                                         |
| Bible           | [MASK]                                              | What is the whole law fulfilled in one word?                  | you shall love your neighbor as yourself                                                               |
| Jimi Hendrix    | [MASK]                                              | In what year did Hendrix headline the Isle of Wight Festival? | 1969                                                                                       |
| Physics Lecture | [MASK]                                              | What is the theory of light quanta?                           | quantum hv capable of yielding all their energy to an electron of the irradiated body |
| Zhuangzi        | [MASK]                                              | Who was the god of the Northern Sea?                          | Hundun                                                                                                 |


It seems again that the unmasked questions are generally better than the masked ones, and the masked questions even had some wrong question-answer pairs (c.f. the Zhuangzi passage). It seems that more work needs to be done to generalize the masked model.

# Limitations and Future Work

A major limitation to our study was the lack of computing power and memory. We could not use the spaCy library due to out-of-memory errors. With that, we could have added weights to different ner and pos tags, and possibly tell that model what type of question we want it to generate, as suggested by Nema, 2018. A "why" question is different from a "who" question, for example. This would mean that we could extend the model to produce inference questions instead of simply comprehension ones, adding a layer of complexity to the generated questions for more advanced learners.

Another major limitation was the scoring metrics. BLEU-n, ROGUE, and METEOR scores were created for machine translation and text summarization, but do not capture the subtleties of question generation, which is a different ball game. Future work could be done on creating a new metric that can capture nuance, context, and interpretability, taking into account grammar and ner tags. This would greatly advance the field of question generation, and ensure that future models would be better but also more flexible in the types of questions they can generate.

Lastly, there has been work done on multilingual question generation models, which can be used for language learners.

# Bibliography

* Kristiyan Vachev, Momchil Hardalov, Georgi Karadzhov, Georgi Georgiev, Ivan Koychev, Preslav Nakov: “Leaf: Multiple-Choice Question Generation”, 2022; http://arxiv.org/abs/2201.09012.

* Colin Raffel, Noam Shazeer, Adam Roberts, Katherine Lee, Sharan Narang, Michael Matena, Yanqi Zhou, Wei Li, Peter J. Liu: “Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer”, 2019; http://arxiv.org/abs/1910.10683.

* Zhe Zhang, Yue Wang, Zhi Tian: “Efficient Two-Dimensional Line Spectrum Estimation Based on Decoupled Atomic Norm Minimization”, 2018; http://arxiv.org/abs/1808.01019.

* Qingyu Zhou, Nan Yang, Furu Wei, Chuanqi Tan, Hangbo Bao, Ming Zhou: “Neural Question Generation from Text: A Preliminary Study”, 2017; http://arxiv.org/abs/1704.0179.

* Pranav Rajpurkar, Jian Zhang, Konstantin Lopyrev, Percy Liang: “SQuAD: 100,000+ Questions for Machine Comprehension of Text”, 2016; http://arxiv.org/abs/1606.05250.

* Chia-Wei Liu, Ryan Lowe, Iulian V. Serban, Michael Noseworthy, Laurent Charlin, Joelle Pineau: “How NOT To Evaluate Your Dialogue System: An Empirical Study of Unsupervised Evaluation Metrics for Dialogue Response Generation”, 2016; http://arxiv.org/abs/1603.08023.

* Chris Callison-Burch, Miles Osborne, and Philipp Koehn. 2006. Re-evaluating the Role of Bleu in Machine Translation Research. In 11th Conference of the European Chapter of the Association for Computational Linguistics, pages 249–256, Trento, Italy. Association for Computational Linguistics.

* Thomas Scialom, Benjamin Piwowarski, and Jacopo Staiano. 2019. Self-Attention Architectures for Answer-Agnostic Neural Question Generation. In Proceedings of the 57th Annual Meeting of the Association for Computational Linguistics, pages 6027–6032, Florence, Italy. Association for Computational Linguistics.